In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

app = Flask(__name__)

# Load the trained model
model = joblib.load('best_model_Linear Regression.pkl')

# Define a function to preprocess input data
def preprocess_data(data):
    # Perform the same preprocessing steps as done before training the model
    columns_to_scale = ['Unadjusted', 'Adjusted_Unadjusted_Difference', 'Adjusted_Unadjusted_Ratio']
    scaler = StandardScaler()
    scaled_data = data.copy()
    scaled_data[columns_to_scale] = scaler.fit_transform(scaled_data[columns_to_scale])
    return scaled_data

# Define the prediction endpoint
@app.route('/', methods=['GET', 'POST'])
def predict_page():
    if request.method == 'POST':
        # Get the input data from the form
        input_data = {}
        input_data['Unadjusted'] = float(request.form['Unadjusted'])
        input_data['Adjusted_Unadjusted_Difference'] = float(request.form['Adjusted_Unadjusted_Difference'])
        input_data['Adjusted_Unadjusted_Ratio'] = float(request.form['Adjusted_Unadjusted_Ratio'])
        
        # Loop through GEO columns
        for geo_column in ['GEO_Alberta', 'GEO_British Columbia', 'GEO_Canada', 'GEO_Manitoba', 'GEO_Montréal, Quebec', 
                           'GEO_New Brunswick', 'GEO_Newfoundland and Labrador', 'GEO_Northwest Territories', 
                           'GEO_Nova Scotia', 'GEO_Nunavut', 'GEO_Ontario', 'GEO_Prince Edward Island', 'GEO_Quebec', 
                           'GEO_Saskatchewan', 'GEO_Toronto, Ontario', 'GEO_Vancouver, British Columbia', 'GEO_Yukon']:
            input_data[geo_column] = 1 if request.form[geo_column] == 'Yes' else 0
        
        # Loop through NAICS columns
        for naics_column in ['North American Industry Classification System (NAICS)_Automobile dealers [4411]', 
                             'North American Industry Classification System (NAICS)_Automotive parts, accessories and tire retailers [4413]', 
                             'North American Industry Classification System (NAICS)_Beer, wine and liquor retailers [4453]', 
                             'North American Industry Classification System (NAICS)_Building material and garden equipment and supplies dealers [444]', 
                             'North American Industry Classification System (NAICS)_Cannabis retailers [459993]', 
                             'North American Industry Classification System (NAICS)_Clothing and clothing accessories retailers [4581]', 
                             'North American Industry Classification System (NAICS)_Clothing, clothing accessories, shoes, jewelry, luggage and leather goods retailers [458]', 
                             'North American Industry Classification System (NAICS)_Convenience retailers and vending machine operators [44513]', 
                             'North American Industry Classification System (NAICS)_Electronics and appliances retailers [4492]', 
                             'North American Industry Classification System (NAICS)_Floor covering, window treatment and other home furnishing retailers [44912]', 
                             'North American Industry Classification System (NAICS)_Food and beverage retailers [445]', 
                             'North American Industry Classification System (NAICS)_Furniture retailers [44911]', 
                             'North American Industry Classification System (NAICS)_Furniture, floor covering, window treatment and other home furnishings retailers [4491]', 
                             'North American Industry Classification System (NAICS)_Furniture, home furnishings, electronics and appliances retailers [449]', 
                             'North American Industry Classification System (NAICS)_Gasoline stations and fuel vendors [457]', 
                             'North American Industry Classification System (NAICS)_General merchandise retailers [455]', 
                             'North American Industry Classification System (NAICS)_Grocery and convenience retailers [4451]', 
                             'North American Industry Classification System (NAICS)_Health and personal care retailers [456]', 
                             'North American Industry Classification System (NAICS)_Jewellery, luggage and leather goods retailers [4583]', 
                             'North American Industry Classification System (NAICS)_Miscellaneous retailers [459B]', 
                             'North American Industry Classification System (NAICS)_Motor vehicle and parts dealers [441]', 
                             'North American Industry Classification System (NAICS)_New car dealers [44111]', 
                             'North American Industry Classification System (NAICS)_Other motor vehicle dealers [4412]', 
                             'North American Industry Classification System (NAICS)_Retail trade [44-45]', 
                             'North American Industry Classification System (NAICS)_Shoe retailers [4582]', 
                             'North American Industry Classification System (NAICS)_Specialty food retailers [4452]', 
                             'North American Industry Classification System (NAICS)_Sporting goods, hobby, musical instrument, book retailers and news dealers [459A]', 
                             'North American Industry Classification System (NAICS)_Sporting goods, hobby, musical instrument, book, and miscellaneous retailers [459]', 
                             'North American Industry Classification System (NAICS)_Supermarkets and other grocery retailers (except convenience retailers) [44511]', 
                             'North American Industry Classification System (NAICS)_Used car dealers [44112]']:
            input_data[naics_column] = 1 if request.form[naics_column] == 'Yes' else 0

        # Convert input data to DataFrame
        input_df = pd.DataFrame([input_data])
        # Preprocess the input data
        preprocessed_data = preprocess_data(input_df)
        # Make predictions using the model
        predictions = model.predict(preprocessed_data)
        # Return predictions as JSON response
        return jsonify({'predictions': predictions.tolist()})
    else:
        # Render HTML form for input
        html_form = """
        <form method="POST">
            <label for="Unadjusted">Unadjusted:</label><br>
            <input type="text" id="Unadjusted" name="Unadjusted" required><br>
            <label for="Adjusted_Unadjusted_Difference">Adjusted Unadjusted Difference:</label><br>
            <input type="text" id="Adjusted_Unadjusted_Difference" name="Adjusted_Unadjusted_Difference" required><br>
            <label for="Adjusted_Unadjusted_Ratio">Adjusted Unadjusted Ratio:</label><br>
            <input type="text" id="Adjusted_Unadjusted_Ratio" name="Adjusted_Unadjusted_Ratio" required><br><br>
        """
        
        # Add dropdowns for GEO columns
        for geo_column in ['GEO_Alberta', 'GEO_British Columbia', 'GEO_Canada', 'GEO_Manitoba', 'GEO_Montréal, Quebec', 
                           'GEO_New Brunswick', 'GEO_Newfoundland and Labrador', 'GEO_Northwest Territories', 
                           'GEO_Nova Scotia', 'GEO_Nunavut', 'GEO_Ontario', 'GEO_Prince Edward Island', 'GEO_Quebec', 
                           'GEO_Saskatchewan', 'GEO_Toronto, Ontario', 'GEO_Vancouver, British Columbia', 'GEO_Yukon']:
            html_form += f'<label for="{geo_column}">{geo_column}:</label>'
            html_form += f'<select name="{geo_column}" id="{geo_column}">'
            html_form += '<option value="0">No</option>'
            html_form += '<option value="1">Yes</option>'
            html_form += '</select><br><br>'
        
        # Add dropdowns for NAICS columns
        for naics_column in ['North American Industry Classification System (NAICS)_Automobile dealers [4411]', 
                             'North American Industry Classification System (NAICS)_Automotive parts, accessories and tire retailers [4413]', 
                             'North American Industry Classification System (NAICS)_Beer, wine and liquor retailers [4453]', 
                             'North American Industry Classification System (NAICS)_Building material and garden equipment and supplies dealers [444]', 
                             'North American Industry Classification System (NAICS)_Cannabis retailers [459993]', 
                             'North American Industry Classification System (NAICS)_Clothing and clothing accessories retailers [4581]', 
                             'North American Industry Classification System (NAICS)_Clothing, clothing accessories, shoes, jewelry, luggage and leather goods retailers [458]', 
                             'North American Industry Classification System (NAICS)_Convenience retailers and vending machine operators [44513]', 
                             'North American Industry Classification System (NAICS)_Electronics and appliances retailers [4492]', 
                             'North American Industry Classification System (NAICS)_Floor covering, window treatment and other home furnishing retailers [44912]', 
                             'North American Industry Classification System (NAICS)_Food and beverage retailers [445]', 
                             'North American Industry Classification System (NAICS)_Furniture retailers [44911]', 
                             'North American Industry Classification System (NAICS)_Furniture, floor covering, window treatment and other home furnishings retailers [4491]', 
                             'North American Industry Classification System (NAICS)_Furniture, home furnishings, electronics and appliances retailers [449]', 
                             'North American Industry Classification System (NAICS)_Gasoline stations and fuel vendors [457]', 
                             'North American Industry Classification System (NAICS)_General merchandise retailers [455]', 
                             'North American Industry Classification System (NAICS)_Grocery and convenience retailers [4451]', 
                             'North American Industry Classification System (NAICS)_Health and personal care retailers [456]', 
                             'North American Industry Classification System (NAICS)_Jewellery, luggage and leather goods retailers [4583]', 
                             'North American Industry Classification System (NAICS)_Miscellaneous retailers [459B]', 
                             'North American Industry Classification System (NAICS)_Motor vehicle and parts dealers [441]', 
                             'North American Industry Classification System (NAICS)_New car dealers [44111]', 
                             'North American Industry Classification System (NAICS)_Other motor vehicle dealers [4412]', 
                             'North American Industry Classification System (NAICS)_Retail trade [44-45]', 
                             'North American Industry Classification System (NAICS)_Shoe retailers [4582]', 
                             'North American Industry Classification System (NAICS)_Specialty food retailers [4452]', 
                             'North American Industry Classification System (NAICS)_Sporting goods, hobby, musical instrument, book retailers and news dealers [459A]', 
                             'North American Industry Classification System (NAICS)_Sporting goods, hobby, musical instrument, book, and miscellaneous retailers [459]', 
                             'North American Industry Classification System (NAICS)_Supermarkets and other grocery retailers (except convenience retailers) [44511]', 
                             'North American Industry Classification System (NAICS)_Used car dealers [44112]']:
            html_form += f'<label for="{naics_column}">{naics_column}:</label>'
            html_form += f'<select name="{naics_column}" id="{naics_column}">'
            html_form += '<option value="0">No</option>'
            html_form += '<option value="1">Yes</option>'
            html_form += '</select><br><br>'
        
        html_form += '<input type="submit" value="Submit">'
        html_form += '</form>'
        
        return html_form

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2024 08:30:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2024 08:31:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2024 08:33:26] "GET / HTTP/1.1" 200 -
